# Sequence Models and Long-Short Term Memory Networks

From the [PyTorch tutorial on sequence models and LSTM](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html):

> At this point, we have seen various feed-forward networks. That is, there is no state maintained by the network at all. This might not be the behavior we want. Sequence models are central to NLP: they are models where there is some sort of dependence through time between your inputs. The classical example of a sequence model is the Hidden Markov Model for part-of-speech tagging. Another example is the conditional random field.

 > A recurrent neural network is a network that maintains some kind of state. For example, its output could be used as part of the next input, so that information can propogate along as the network passes over the sequence. In the case of an LSTM, for each element in the sequence, there is a corresponding *hidden state* $h_t$, which in principle can contain information from arbitrary points earlier in the sequence. We can use the hidden state to predict words in a language model, part-of-speech tags, and a myriad of other things.
 
 ## LSTMs in PyTorch
 
PyTorch's LSTM expects 3D tensors of shape (*seq_len, batch, input_size*). For example, in the WordEmbeddings notebook we used an embedding layer with 128-dimensional embeddings. If we had input sequences of length 10, and a minibatch size of 64, the input tensor to the LSTM would have size (10, 64, 128).

In [1]:
%matplotlib inline

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt

torch.manual_seed(1)

# Simple example of an LSTM
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]    # A 5-long sequence of 1 x 3 tensors

# Initialize the hidden state
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time. After each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    
# Alternatively, we can do the entire sequence at once.
# The first value returned by LSTM is all of the hidden states throughout the sequence. The
# second is just the most recent hidden state (compare the last slice of "out" with "hidden"
# below - they are the same)
# The reason for this is that: 
# "out" will give you access to all hidden states in the sequence.
# "hidden" will allow you to continue the sequence and backpropogate, by passing it as an 
# argument to the LSTM at a later time
# Add the second dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))   # Clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]])
(tensor([[[-0.3368,  0.0959, -0.0538]]]), tensor([[[-0.9825,  0.4715, -0.0633]]]))


## LSTM for part-of-speech tagging

Now we'll design a model that uses an LSTM to get part-of-speech tags (like "verb", "adjective", etc).

Let our input sequence be $w_1, ... , w_M$, where $w_i \in V$, our vocabulary. We define $T$ to be the tag set and $y_i$ to be the acutal tag of a word $w_i$. The predicted tag of $w_i$ is $\hat{y}_i$.

This is a structure prediction, model, where our output is a sequence $\hat{y}_1, ... , \hat{y}_M$, where $\hat{y}_i \in T$.

To do the prediction, we pass the sentence into an LSTM network. The hidden state at each timestep $i$ is written as $h_i$, and each tag is given a unique index (like `word_to_idx` in the WordEmbedding notebook). 

The prediction rule for the predicted tag $\hat{y}_i$ is:

<center>$\hat{y}_i = \text{argmax}_j (\log \text{Softmax}(Ah_i + b))_j$</center>

That is, take the log softmax of the affine map of the hidden state, and the predicted tag is the tag with the highest score in the resulting logits vector. This implies that the dimensionality of the target space of $A$ is $|T|$.

In [2]:
def prepare_sequence(seq, to_idx):
    idxs = [to_idx[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_idx = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)
print(word_to_idx)
tag_to_idx = {"DET": 0, "NN": 1, "V": 2}
idx_to_tag = {v: k for k, v in tag_to_idx.items()}

# These will usually be more like 32 or 64 dimensional.
# We'll keep them small so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


Create the model:

In [3]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # Before we've done anything, we don't have any hidden state. Refer to the
        # PyTorch documentation to see exactly why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Train the model:

In [4]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_idx), len(tag_to_idx))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i, j of the output is the score for tag j and word i.
# We don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_idx)
    tag_scores = model(inputs)
    print(tag_scores)
    
# Train for real
for epoch in range(300):     # Wouldn't normally do so many epochs
    for sentence, tags in training_data:
        # Step 1. Remember that PyTorch accumulates gradients.
        # Need to clear them out before each instance
        model.zero_grad()
        
        # Clear out hidden state of LSTM, detaching it from its history on
        # the last iteration
        model.hidden = model.init_hidden()
        
        # Step 2. Get inputs ready for the network (list -> Tensor of word indices)
        sentence_in = prepare_sequence(sentence, word_to_idx)
        targets = prepare_sequence(tags, tag_to_idx)
        
        # Step 3. Run forward pass
        tag_scores = model(sentence_in)
        
        # Step 4. Compute the loss, gradients, and update the parameters by calling
        # optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_idx)
    tag_scores = model(inputs)
    
    # The sentence is "the dog ate the apple". i, j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag. Here, we can see the
    # predicted sequence below is 0, 1, 2, 0, 1, which is DET NOUN VERB DET NOUN
    print("tag scores:\n{}".format(tag_scores))
    predicted_tags = np.argmax(tag_scores.detach().numpy(), axis=1)
    print([idx_to_tag[i] for i in predicted_tags.tolist()])

tensor([[-1.1389, -1.2024, -0.9693],
        [-1.1065, -1.2200, -0.9834],
        [-1.1286, -1.2093, -0.9726],
        [-1.1190, -1.1960, -0.9916],
        [-1.0137, -1.2642, -1.0366]])
tag scores:
tensor([[-0.0858, -2.9355, -3.5374],
        [-5.2313, -0.0234, -4.0314],
        [-3.9098, -4.1279, -0.0368],
        [-0.0187, -4.7809, -4.5960],
        [-5.8170, -0.0183, -4.1879]])
['DET', 'NN', 'V', 'DET', 'NN']


## Augmenting the LSTM part-of-speech tagger with character-level features

From the associated tutorial:

> In the example above, each word had an embedding, which served as the inputs to our sequence model. Let’s augment the word embeddings with a representation derived from the characters of the word. We expect that this should help significantly, since character-level information like affixes have a large bearing on part-of-speech. For example, words with the affix *-ly* are almost always tagged as adverbs in English.
> 
> To do this, let $c_w$ be the character-level representation of word $w$. Let $x_w$ be the word embedding as before. Then the input to our sequence model is the concatenation of $x_w$ and $c_w$. So if $x_w$ has dimension 5, and $c_w$ dimension 3, then our LSTM should accept an input of dimension 8.
>
> To get the character level representation, do an LSTM over the characters of a word, and let $c_w$ be the final hidden state of this LSTM. Hints:
> - There are going to be two LSTM’s in your new model. The original one that outputs POS tag scores, and the new one that outputs a character-level representation of each word.
> - To do a sequence model over characters, you will have to embed characters. The character embeddings will be the input to the character LSTM.

In [5]:
# convert the characters of each word into a usable data set
char_to_idx = {}
for sentence, _ in training_data:
    for word in sentence:
        for char in list(word):
            if char not in char_to_idx:
                char_to_idx[char] = len(char_to_idx) + 1
print(char_to_idx)
    
def prepare_data(sentence, word_to_idx, char_to_idx):
    word_idxs = [word_to_idx[w] for w in sentence]
    word_tensors = torch.tensor(word_idxs, dtype=torch.long)
    
    # Create tensor for characters
    char_tensors = []
    for word in sentence:
        char_list = [char_to_idx[c] for c in list(word)]
        char_tensors.append(torch.tensor(char_list, dtype=torch.long))
    # Sort tensors in descending order
    word_lengths = [-len(t) for t in char_tensors]
    sort_idxs = np.argsort(word_lengths)
    # Create 2D tensor of character indices by padding
    padded_chars = nn.utils.rnn.pad_sequence([char_tensors[i] for i in sort_idxs])
    
    return word_tensors, padded_chars, sort_idxs
    
# Test this
word_tensors, char_tensors, unsort_map = prepare_data(training_data[1][0], word_to_idx, char_to_idx)
print(word_tensors)
print(char_tensors)
print(unsort_map)

{'T': 1, 'h': 2, 'e': 3, 'd': 4, 'o': 5, 'g': 6, 'a': 7, 't': 8, 'p': 9, 'l': 10, 'E': 11, 'v': 12, 'r': 13, 'y': 14, 'b': 15, 'k': 16}
tensor([ 5,  6,  7,  8])
tensor([[ 11,  13,   8,  15],
        [ 12,   3,   2,   5],
        [  3,   7,   7,   5],
        [ 13,   4,   8,  16],
        [ 14,   0,   0,   0],
        [ 15,   0,   0,   0],
        [  5,   0,   0,   0],
        [  4,   0,   0,   0],
        [ 14,   0,   0,   0]])
[0 1 2 3]


In [6]:
class CharLvlTagger(nn.Module):
    def __init__(self, word_embedding_dim, 
                 char_embedding_dim, 
                 word_hidden_dim, 
                 char_hidden_dim, 
                 vocab_size, 
                 num_chars, 
                 tagset_size):
        super(CharLvlTagger, self).__init__()
        # General local vars
        self.char_hidden_dim = char_hidden_dim
        self.word_hidden_dim = word_hidden_dim
        
        # Word embedding layer
        self.word_embeddings = nn.Embedding(vocab_size, word_embedding_dim)
        # Char embedding layer
        self.char_embeddings = nn.Embedding(num_chars, char_embedding_dim)
        
        # First LSTM layer, where we build a representation of the characters
        self.char_lstm = nn.LSTM(char_embedding_dim, char_hidden_dim)
        # Second LSTM layer. The input is the concatenation of the word embedding and the
        # character-level representation of the word
        self.word_lstm = nn.LSTM(word_embedding_dim + char_hidden_dim, word_hidden_dim)
        
        # Affine layer mapping wordvecs to logits
        self.hidden2tag = nn.Linear(word_hidden_dim, tagset_size)
        
        # Initial hidden layers
        self.char_hidden = self.init_hidden(1, char_hidden_dim)
        self.word_hidden = self.init_hidden(1, word_hidden_dim)
        
    def init_hidden(self, batch_sz, hidden_dim):
        # Zeroize the hidden state
        return (torch.zeros(1, batch_sz, hidden_dim), 
                torch.zeros(1, batch_sz, hidden_dim))
    
    def forward(self, words, chars, unsort_map):
        # Zero hidden states
        self.char_hidden = self.init_hidden(len(words), self.char_hidden_dim)
        self.word_hidden = self.init_hidden(1, self.word_hidden_dim)
        
        # Get word and character embeddings
        word_embeds = self.word_embeddings(words)
        char_embeds = self.char_embeddings(chars)
        
        # Run character embeddings through LSTM
        char_lstm_out, self.char_hidden = self.char_lstm(char_embeds, self.char_hidden)
        #hidden_out = torch.gather(torch.squeeze(self.char_hidden[0], dim=0), 
        #                          dim=0, 
        #                          index=torch.tensor(unsort_map))
        hidden_out = torch.squeeze(self.char_hidden[0], dim=0)
        hidden_out = hidden_out[unsort_map, :]
        # Concatenate the output hidden layer of the character-level LSTM to the word representations
        word_char = torch.cat([word_embeds, hidden_out], dim=1).unsqueeze(1)
        word_lstm_out, self.word_hidden = self.word_lstm(word_char, self.word_hidden)
        
        # Affine map to word tag space
        tag_space = self.hidden2tag(word_lstm_out.view(len(words), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Train the model:

In [7]:
WORD_EMBEDDING_DIM = 32
CHAR_EMBEDDING_DIM = 32
WORD_HIDDEN_DIM = 6
CHAR_HIDDEN_DIM = 3

model = CharLvlTagger(WORD_EMBEDDING_DIM, 
                      CHAR_EMBEDDING_DIM, 
                      WORD_HIDDEN_DIM, 
                      CHAR_HIDDEN_DIM, 
                      len(word_to_idx), 
                      len(char_to_idx) + 1,  # Necessary since we zero-padded unequal length sequences 
                      len(tag_to_idx))
loss_fn = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
with torch.no_grad():
    word_tensors, char_tensors, unsort_map = prepare_data(training_data[0][0], 
                                                          word_to_idx, 
                                                          char_to_idx)
    tag_scores = model(word_tensors, char_tensors, unsort_map)
    print(tag_scores)
    
# Train for real
for epoch in range(300):
    for sentence, tags in training_data:
        model.zero_grad()
        word_tensors, char_tensors, unsort_map = prepare_data(sentence, word_to_idx, char_to_idx)
        targets = prepare_sequence(tags, tag_to_idx)
        
        # Forward pass
        tag_scores = model(word_tensors, char_tensors, unsort_map)
        
        # Compute loss, gradients, and update params
        loss = loss_fn(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
# See what the scores are after training
with torch.no_grad():
    word_tensors, char_tensors, unsort_map = prepare_data(training_data[0][0], 
                                                          word_to_idx, 
                                                          char_to_idx)
    tag_scores = model(word_tensors, char_tensors, unsort_map)
    print("Tag scores:\n{}".format(tag_scores))
    predicted_tags = np.argmax(tag_scores.detach().numpy(), axis=1)
    print("Predicted tags:\n{}".format([idx_to_tag[i] for i in predicted_tags.tolist()]))

tensor([[-1.4144, -1.2138, -0.7768],
        [-1.3311, -1.2105, -0.8261],
        [-1.2147, -1.3579, -0.8074],
        [-1.3896, -1.1621, -0.8255],
        [-1.2954, -1.2081, -0.8499]])
Tag scores:
tensor([[-0.0214, -4.5017, -4.5949],
        [-5.5343, -0.0139, -4.6232],
        [-3.0364, -3.3154, -0.0881],
        [-0.0430, -3.9105, -3.8155],
        [-4.0689, -0.0221, -5.3535]])
Predicted tags:
['DET', 'NN', 'V', 'DET', 'NN']
